# PySpark Toolkit

In [42]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

### Use case : Data Profiling 

In [44]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)), # Totally duplicate
    Row(a=4, b=5., c='string3', d=date(2000, 5, 1), e=datetime(2000, 1, 3, 12, 0)), # Part duplicate
    Row(a=None, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)), # PK contains Null
    Row(a=6, b=None, c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)) # PK contains Null
])
df.show()

+----+----+-------+----------+-------------------+
|   a|   b|      c|         d|                  e|
+----+----+-------+----------+-------------------+
|   1| 2.0|string1|2000-01-01|2000-01-01 12:00:00|
|   2| 3.0|string2|2000-02-01|2000-01-02 12:00:00|
|   4| 5.0|string3|2000-03-01|2000-01-03 12:00:00|
|   4| 5.0|string3|2000-03-01|2000-01-03 12:00:00|
|   4| 5.0|string3|2000-05-01|2000-01-03 12:00:00|
|null| 5.0|string3|2000-03-01|2000-01-03 12:00:00|
|   6|null|string3|2000-03-01|2000-01-03 12:00:00|
+----+----+-------+----------+-------------------+



In [52]:
PK = ['a','b','c']

In [6]:
# PK null check
from pyspark.sql.functions import lit
null_check_str = ' is null or '.join(PK) + ' is null'
df_null = df.filter(null_check_str)
df_null = df_null.withColumn('desc',lit('PK Contains Null'))
df_null.show()

+----+----+-------+----------+-------------------+----------------+
|   a|   b|      c|         d|                  e|            desc|
+----+----+-------+----------+-------------------+----------------+
|null| 5.0|string3|2000-03-01|2000-01-03 12:00:00|PK Contains Null|
|   6|null|string3|2000-03-01|2000-01-03 12:00:00|PK Contains Null|
+----+----+-------+----------+-------------------+----------------+



In [7]:
from pyspark.sql.functions import count


In [61]:
from pyspark.sql.functions import col
df_dups = df.groupby(PK).count().filter('count>1')

df_dups = df_dups.select(PK+["count"])
df_dups.show()

+---+---+-------+-----+
|  a|  b|      c|count|
+---+---+-------+-----+
|  4|5.0|string3|    3|
+---+---+-------+-----+



In [68]:
df.join(df_dups,on=PK,how='left').where('count is null').select(PK).show()


+----+----+-------+
|   a|   b|      c|
+----+----+-------+
|   1| 2.0|string1|
|   2| 3.0|string2|
|null| 5.0|string3|
|   6|null|string3|
+----+----+-------+



In [69]:
def pk_null_check(df,):
    
    return df_null

def pk_dups_check(df):
    
    return df_dups

def profiling_data(df,PK = []):
    
    return df_clean,df_reject
    

### Use case : Data Sampling

### Use case : Row to Column